In [ ]:
import openai
import json

# OpenAI API Key (Replace with your actual key)
openai.api_key = "your-api-key"

# Mock function for flight details
def get_flight_info(origin, destination):
    return {
        "origin": origin,
        "destination": destination,
        "flight_number": "KL643",
        "airline": "KLM",
        "departure_time": "2025-03-15 10:30 AM"
    }

# Mock function for hotel details
def get_hotel_info(city):
    return {
        "city": city,
        "hotel_name": "Grand Hotel",
        "price_per_night": "$150",
        "rating": "4.5 stars"
    }

# Define function descriptions for OpenAI
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Fetches flight details based on origin and destination.",
        "parameters": {
            "type": "object",
            "properties": {
                "origin": {"type": "string", "description": "Departure airport code (e.g., JFK)"},
                "destination": {"type": "string", "description": "Arrival airport code (e.g., LAX)"}
            },
            "required": ["origin", "destination"]
        }
    },
    {
        "name": "get_hotel_info",
        "description": "Fetches hotel details for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name for hotel search"}
            },
            "required": ["city"]
        }
    }
]

def ai_assistant(user_prompt):
    # Call OpenAI API with function calling
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": user_prompt}],
        functions=function_descriptions,
        function_call="auto"
    )

    # Check if the model decided to call a function
    if response["choices"][0]["message"].get("function_call"):
        function_name = response["choices"][0]["message"]["function_call"]["name"]
        arguments = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])

        # Call the corresponding function
        if function_name == "get_flight_info":
            result = get_flight_info(**arguments)
        elif function_name == "get_hotel_info":
            result = get_hotel_info(**arguments)

        return {"function_called": function_name, "result": result}

    return {"message": "No function call was needed."}

# Example user inputs
user_queries = [
    "I need a flight from JFK to LAX.",
    "Find me a hotel in Paris."
]

# Run the assistant for each query
for query in user_queries:
    response = ai_assistant(query)
    print(f"User Query: {query}")
    print(f"Response: {response}")
    print("-" * 50)
